<a href="https://colab.research.google.com/github/clausM2/human_pose/blob/main/Using_VideoPose3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise Form Checker Using Pose Estimation by VideoPose3D

# Install Detectron2





In [4]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 18.4MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=91c529b5a07815fd8c9cb3025ec4241a276e96425d15c6779512629bde5a3b07
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.8.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [5]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.2MB 644kB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 2.2MB 17.8MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-cp37-none-any.whl size=58543 sha256=334e68b2811ac4961a8222054939c3e1292d7e6a7531239dd187714b782b783a
  Stored in directory: /root/.cache/pip/wheels/d2/ee/3a/5c531df777c03d8c67f22c65f97d6f75321087482d05a9b218
  Created wheel for iopath: filename=iopath-0.1.6-cp37-none-any.whl size=18269 sha256=d22daee93ef30050448be2984a39ef5a189cf3239bbafb545db7b7656fb42200
  Stored in directory: /root/.cache/pip/wheels/ff/87/55/7513122394822977f993e1798bfebb46730d6183df8db7b283
Successfully built fvcore iopath
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling 

Clone the VideoPose3D repo to have access to utility algorithms



In [16]:
!git clone https://github.com/facebookresearch/VideoPose3D.git


Cloning into 'VideoPose3D'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121
Receiving objects: 100% (121/121), 9.53 MiB | 44.96 MiB/s, done.
Resolving deltas: 100% (41/41), done.


# Infer keypoints from the mp4 videos in the vid directory 





In [34]:
!cd inference && python infer_video_d2.py --cfg 'COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml' --output-dir output_directory --image-ext mp4 vid

Processing vid/menshealth_facingleft.mp4
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --ena

#Run dataset preproceessing script to create a custom dataset





In [35]:
!cd data && python prepare_data_2d_custom.py -i ../inference/output_directory -o myvideos

Parsing 2D detections from ../inference/output_directory
Processing ../inference/output_directory/mindpump_left.mp4.npz
71 total frames processed
0 frames were interpolated
----------
Processing ../inference/output_directory/startingstrength_right.mp4.npz
40 total frames processed
0 frames were interpolated
----------
Processing ../inference/output_directory/athlean1.mp4.npz
258 total frames processed
0 frames were interpolated
----------
Processing ../inference/output_directory/martinlicis_facingleft.mp4.npz
66 total frames processed
0 frames were interpolated
----------
Processing ../inference/output_directory/stoppani_forward.mp4.npz
83 total frames processed
0 frames were interpolated
----------
Processing ../inference/output_directory/jeffnipard_facingleft.mp4.npz
52 total frames processed
0 frames were interpolated
----------
Processing ../inference/output_directory/ryan_humiston_facingright.mp4.npz
77 total frames processed
0 frames were interpolated
----------
Processing ../inf

# Export coordinates and custom videos 





In [58]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject jeffnipard_facingleft.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/jeffnipard_facingleft.mp4 --viz-output o_jeff.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_jacob.mp4', viz_size=6, viz_skip=0, viz_subject='jacobalava_facingleft.mp4', viz_video='inference/vid/jacobalava_facingleft.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INF

In [59]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject jeffnipard_facingleft.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/jeffnipard_facingleft.mp4 --viz-output o_jeff.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_jeff.mp4', viz_size=6, viz_skip=0, viz_subject='jeffnipard_facingleft.mp4', viz_video='inference/vid/jeffnipard_facingleft.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO

In [60]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject martinlicis_facingleft.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/martinlicis_facingleft.mp4 --viz-output o_martin.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_martin.mp4', viz_size=6, viz_skip=0, viz_subject='martinlicis_facingleft.mp4', viz_video='inference/vid/martinlicis_facingleft.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...


In [62]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject martinlicis_facingleft.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/martinlicis_facingleft.mp4 --viz-export o_martin --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_martin', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='martinlicis_facingleft.mp4', viz_video='inference/vid/martinlicis_facingleft.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO

In [63]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject jeffnipard_facingleft.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/jeffnipard_facingleft.mp4 --viz-export o_jeff --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_jeff', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='jeffnipard_facingleft.mp4', viz_video='inference/vid/jeffnipard_facingleft.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Re

In [64]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject menshealth_facingleft.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/menshealth_facingleft.mp4 --viz-output o_men.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_men.mp4', viz_size=6, viz_skip=0, viz_subject='menshealth_facingleft.mp4', viz_video='inference/vid/menshealth_facingleft.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO:

In [65]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject menshealth_facingleft.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/menshealth_facingleft.mp4 --viz-export o_men --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_men', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='menshealth_facingleft.mp4', viz_video='inference/vid/menshealth_facingleft.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Rec

In [67]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject mikethurston_right.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/mikethurston_right.mp4 --viz-output o_mike.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_mike.mp4', viz_size=6, viz_skip=0, viz_subject='mikethurston_right.mp4', viz_video='inference/vid/mikethurston_right.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Rece

In [68]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject mikethurston_right.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/mikethurston_right.mp4 --viz-export o_mike --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_mike', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='mikethurston_right.mp4', viz_video='inference/vid/mikethurston_right.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptiv

In [69]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject mindpump_left.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/mindpump_left.mp4 --viz-output o_mind.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_mind.mp4', viz_size=6, viz_skip=0, viz_subject='mindpump_left.mp4', viz_video='inference/vid/mindpump_left.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive fiel

In [70]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject mindpump_left.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/mindpump_left.mp4 --viz-export o_mind --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_mind', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='mindpump_left.mp4', viz_video='inference/vid/mindpump_left.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 2

In [71]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject ryan_humiston_facingright.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/ryan_humiston_facingright.mp4 --viz-output o_ryanh.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_ryanh.mp4', viz_size=6, viz_skip=0, viz_subject='ryan_humiston_facingright.mp4', viz_video='inference/vid/ryan_humiston_facingright.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detection

In [72]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject ryan_humiston_facingright.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/ryan_humiston_facingright.mp4 --viz-export o_ryanh --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_ryanh', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='ryan_humiston_facingright.mp4', viz_video='inference/vid/ryan_humiston_facingright.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...

In [73]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject scottherman_facingright.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/scottherman_facingright.mp4 --viz-output o_scott.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_scott.mp4', viz_size=6, viz_skip=0, viz_subject='scottherman_facingright.mp4', viz_video='inference/vid/scottherman_facingright.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...

In [74]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject scottherman_facingright.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/scottherman_facingright.mp4 --viz-export o_scott --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_scott', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='scottherman_facingright.mp4', viz_video='inference/vid/scottherman_facingright.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INF

In [75]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject startingstrength_right.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/startingstrength_right.mp4 --viz-output o_start.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_start.mp4', viz_size=6, viz_skip=0, viz_subject='startingstrength_right.mp4', viz_video='inference/vid/startingstrength_right.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
I

In [76]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject startingstrength_right.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/startingstrength_right.mp4 --viz-export o_start --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_start', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='startingstrength_right.mp4', viz_video='inference/vid/startingstrength_right.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO:

In [77]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject stoppani_forward.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/stoppani_forward.mp4 --viz-output o_stopani.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_stopani.mp4', viz_size=6, viz_skip=0, viz_subject='stoppani_forward.mp4', viz_video='inference/vid/stoppani_forward.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Recep

In [78]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject stoppani_forward.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/stoppani_forward.mp4 --viz-export o_stopani --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_stopani', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='stoppani_forward.mp4', viz_video='inference/vid/stoppani_forward.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive

In [79]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject yatinder-right.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/yatinder-right.mp4 --viz-output o_yatinder.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_yatinder.mp4', viz_size=6, viz_skip=0, viz_subject='yatinder-right.mp4', viz_video='inference/vid/yatinder-right.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptiv

In [80]:
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject yatinder-right.mp4 --viz-action custom --viz-camera 0 --viz-video inference/vid/yatinder-right.mp4 --viz-export o_yatinder --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_yatinder', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='yatinder-right.mp4', viz_video='inference/vid/yatinder-right.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive fi

In [81]:
!cd inference && python infer_video_d2.py --cfg 'COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml' --output-dir moreoutput --image-ext mp4 morevid

Processing morevid/Ethier_Deadlift-Proper1.mp4
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265

In [82]:
!cd data && python prepare_data_2d_custom.py -i ../inference/moreoutput -o moreoutvid

Parsing 2D detections from ../inference/moreoutput
Processing ../inference/moreoutput/dead4.mp4.npz
62 total frames processed
0 frames were interpolated
----------
Processing ../inference/moreoutput/dead11.mp4.npz
37 total frames processed
0 frames were interpolated
----------
Processing ../inference/moreoutput/Ethier_Deadlift-Mistake2.mp4.npz
271 total frames processed
0 frames were interpolated
----------
Processing ../inference/moreoutput/Ethier_Deadlift-Mistake1.mp4.npz
105 total frames processed
0 frames were interpolated
----------
Processing ../inference/moreoutput/dead6.mp4.npz
45 total frames processed
0 frames were interpolated
----------
Processing ../inference/moreoutput/dead2.mp4.npz
65 total frames processed
0 frames were interpolated
----------
Processing ../inference/moreoutput/dead1.mp4.npz
65 total frames processed
0 frames were interpolated
----------
Processing ../inference/moreoutput/Ethier_Deadlift-Proper0.mp4.npz
375 total frames processed
0 frames were interpola

In [83]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Deadlift_Front_Test.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Deadlift_Front_Test.mp4 --viz-output o_d11.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_d11.mp4', viz_size=6, viz_skip=0, viz_subject='Deadlift_Front_Test.mp4', viz_video='inference/morevid/Deadlift_Front_Test.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INF

In [84]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Deadlift_Front_Test.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Deadlift_Front_Test.mp4 --viz-export o_d11 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_d11', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='Deadlift_Front_Test.mp4', viz_video='inference/morevid/Deadlift_Front_Test.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: R

In [85]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Deadlift_Right.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Deadlift_Right.mp4 --viz-output o_d12.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_d12.mp4', viz_size=6, viz_skip=0, viz_subject='Deadlift_Right.mp4', viz_video='inference/morevid/Deadlift_Right.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Recepti

In [86]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Deadlift_Right.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Deadlift_Right.mp4 --viz-export o_d12 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_d12', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='Deadlift_Right.mp4', viz_video='inference/morevid/Deadlift_Right.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive f

In [87]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Mistake0.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Mistake0.mp4 --viz-output o_ethier_mistake0.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_ethier_mistake0.mp4', viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Mistake0.mp4', viz_video='inference/morevid/Ethier_Deadlift-Mistake0.mp4', warmup=1)
Loading dataset...
Preparing data...
Loadin

In [88]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Mistake0.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Mistake0.mp4 --viz-export o_ethier_mistake0 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_ethier_mistake0', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Mistake0.mp4', viz_video='inference/morevid/Ethier_Deadlift-Mistake0.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D

In [89]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Mistake1.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Mistake1.mp4 --viz-output o_ethier_mistake1.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_ethier_mistake1.mp4', viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Mistake1.mp4', viz_video='inference/morevid/Ethier_Deadlift-Mistake1.mp4', warmup=1)
Loading dataset...
Preparing data...
Loadin

In [90]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Mistake1.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Mistake1.mp4 --viz-export o_ethier_mistake1 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_ethier_mistake1', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Mistake1.mp4', viz_video='inference/morevid/Ethier_Deadlift-Mistake1.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D

In [91]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Mistake2.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Mistake2.mp4 --viz-output o_ethier_mistake2.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_ethier_mistake2.mp4', viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Mistake2.mp4', viz_video='inference/morevid/Ethier_Deadlift-Mistake2.mp4', warmup=1)
Loading dataset...
Preparing data...
Loadin

In [92]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Mistake2.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Mistake2.mp4 --viz-export o_ethier_mistake2 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_ethier_mistake2', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Mistake2.mp4', viz_video='inference/morevid/Ethier_Deadlift-Mistake2.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D

In [93]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Mistake3.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Mistake3.mp4 --viz-output o_ethier_mistake3.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_ethier_mistake3.mp4', viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Mistake3.mp4', viz_video='inference/morevid/Ethier_Deadlift-Mistake3.mp4', warmup=1)
Loading dataset...
Preparing data...
Loadin

In [94]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Mistake3.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Mistake3.mp4 --viz-export o_ethier_mistake3 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_ethier_mistake3', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Mistake3.mp4', viz_video='inference/morevid/Ethier_Deadlift-Mistake3.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D

In [95]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Proper0-1.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Proper0-1.mp4 --viz-output o_ethier_proper01.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_ethier_proper01.mp4', viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Proper0-1.mp4', viz_video='inference/morevid/Ethier_Deadlift-Proper0-1.mp4', warmup=1)
Loading dataset...
Preparing data...
Load

In [96]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Proper0-1.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Proper0-1.mp4 --viz-export o_ethier_proper01 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_ethier_proper01', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Proper0-1.mp4', viz_video='inference/morevid/Ethier_Deadlift-Proper0-1.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 

In [97]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Proper0.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Proper0.mp4 --viz-output o_ethier_proper0.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_ethier_proper0.mp4', viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Proper0.mp4', viz_video='inference/morevid/Ethier_Deadlift-Proper0.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2

In [98]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Proper0.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Proper0.mp4 --viz-export o_ethier_proper0 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_ethier_proper0', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Proper0.mp4', viz_video='inference/morevid/Ethier_Deadlift-Proper0.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D de

In [99]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Proper1.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Proper1.mp4 --viz-output o_ethier_proper1.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_ethier_proper1.mp4', viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Proper1.mp4', viz_video='inference/morevid/Ethier_Deadlift-Proper1.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2

In [100]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Proper1.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Proper1.mp4 --viz-export o_ethier_proper1 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_ethier_proper1', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Proper1.mp4', viz_video='inference/morevid/Ethier_Deadlift-Proper1.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D de

In [102]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Proper4.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Proper4.mp4 --viz-output o_ethier_proper4.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='o_ethier_proper4.mp4', viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Proper4.mp4', viz_video='inference/morevid/Ethier_Deadlift-Proper4.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2

In [103]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject Ethier_Deadlift-Proper4.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/Ethier_Deadlift-Proper4.mp4 --viz-export o_ethier_proper4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='o_ethier_proper4', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='Ethier_Deadlift-Proper4.mp4', viz_video='inference/morevid/Ethier_Deadlift-Proper4.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D de

In [104]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead1.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead1.mp4 --viz-output d1.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='d1.mp4', viz_size=6, viz_skip=0, viz_subject='dead1.mp4', viz_video='inference/morevid/dead1.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames


In [105]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead1.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead1.mp4 --viz-export d1 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='d1', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='dead1.mp4', viz_video='inference/morevid/dead1.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO

In [106]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead2.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead2.mp4 --viz-output d2.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='d2.mp4', viz_size=6, viz_skip=0, viz_subject='dead2.mp4', viz_video='inference/morevid/dead2.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames


In [107]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead2.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead2.mp4 --viz-export d2 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='d2', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='dead2.mp4', viz_video='inference/morevid/dead2.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO

In [108]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead3.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead3.mp4 --viz-output d3.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='d3.mp4', viz_size=6, viz_skip=0, viz_subject='dead3.mp4', viz_video='inference/morevid/dead3.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames


In [109]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead3.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead3.mp4 --viz-export d3 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='d3', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='dead3.mp4', viz_video='inference/morevid/dead3.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO

In [110]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead4.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead4.mp4 --viz-output d4.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='d4.mp4', viz_size=6, viz_skip=0, viz_subject='dead4.mp4', viz_video='inference/morevid/dead4.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames


In [111]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead4.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead4.mp4 --viz-export d4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='d4', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='dead4.mp4', viz_video='inference/morevid/dead4.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO

In [112]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead5.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead5.mp4 --viz-output d5.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='d5.mp4', viz_size=6, viz_skip=0, viz_subject='dead5.mp4', viz_video='inference/morevid/dead5.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames


In [113]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead5.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead5.mp4 --viz-export d5.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='d5.mp4', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='dead5.mp4', viz_video='inference/morevid/dead5.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames


In [114]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead6.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead6.mp4 --viz-output d6.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='d6.mp4', viz_size=6, viz_skip=0, viz_subject='dead6.mp4', viz_video='inference/morevid/dead6.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames


In [115]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead6.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead6.mp4 --viz-export d6 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='d6', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='dead6.mp4', viz_video='inference/morevid/dead6.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO

In [116]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead7.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead7.mp4 --viz-output d7.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='d7.mp4', viz_size=6, viz_skip=0, viz_subject='dead7.mp4', viz_video='inference/morevid/dead7.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames


In [117]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead7.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead7.mp4 --viz-export d7 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='d7', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='dead7.mp4', viz_video='inference/morevid/dead7.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO

In [118]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead8.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead8.mp4 --viz-output d8.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='d8.mp4', viz_size=6, viz_skip=0, viz_subject='dead8.mp4', viz_video='inference/morevid/dead8.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames


In [119]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead8.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead8.mp4 --viz-export d8 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='d8', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='dead8.mp4', viz_video='inference/morevid/dead8.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO

In [120]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead9.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead9.mp4 --viz-output d9.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='d9.mp4', viz_size=6, viz_skip=0, viz_subject='dead9.mp4', viz_video='inference/morevid/dead9.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames


In [121]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead9.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead9.mp4 --viz-export d9 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='d9', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='dead9.mp4', viz_video='inference/morevid/dead9.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO

In [122]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead10.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead10.mp4 --viz-output d10.mp4 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output='d10.mp4', viz_size=6, viz_skip=0, viz_subject='dead10.mp4', viz_video='inference/morevid/dead10.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 fram

In [123]:
!python run.py -d custom -k moreoutvid -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject dead10.mp4 --viz-action custom --viz-camera 0 --viz-video inference/morevid/dead10.mp4 --viz-export d10 --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='moreoutvid', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='d10', viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=6, viz_skip=0, viz_subject='dead10.mp4', viz_video='inference/morevid/dead10.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
I